<h1 align="center">Reporte Módulo 2: Cómputo matricial</h1> 

<h2 align="center">Singular Value Decomposition on GPU using CUDA</h2>

Existen muy diversas aplicaciones para la descomposición en valores singulares de una matriz densa, desde procesamiento de señales, hasta solución de ecuaciones lineales homogéneas, cálculo de matrices pseudoinversas y análisis de componentes principales.
Más allá de discutir las aplicaciones que tiene esta descomposición, en este reporte se presentará a manera de implementar el algoritmo de cálculo de la misma mediante un paradigma en paralelo utilizando la tecnología CUDA de NVIDIA implementada por Sheetal Lahabar y P. J. Narayanan.

### Descomposición en valores singulares:

La descomposición en valores singulares de una matriz consiste en factorizar la matriz $ A \in {\rm I\!R}^{m x n} $ de la forma:

<h4 align="center">$ A = UΣV^{T} $</h4> 

Donde $ U \in {\rm I\!R}^{m x m} $ is una matriz ortogonal, $ V \in {\rm I\!R}^{n x n} $ es ortogonal y $Σ \in {\rm I\!R}^{m x n}$ es una matriz diagonal con elementos mayores a cero.

### NVIDIA CUDA

Las GPU´s implementadas por NVIDIA proveen interfaces desarrolladas y compatibles con diversos lenguajes de programación tanto de bajo como de alto nivel, de forma que con distintos niveles de accesibilidad y detalle pueden desarrollarse trabajos y metodologías CUDA en C y Python.

En este caso, para la implementacion de un algoritmo de descomposición en valores singulares (SVD) se utilizarán las librerías CUBLAS y CUDA kernels.

Existen algunos esfuerzos precedentes para la implementación de algoritmos algebraicos en un entorno de GPU's, tales como esfuerzos para factorización directa, resolución de ecuaciones lineales y no lineales, así como optimización por puntos interiores. Para el caso de la descomposición en valores singulares esta será la primera vez que se implemente en CUDA.

### Algoritmo

Se realizará mediante el método Golub - Reinsch (Bidiagonalización y diagonalización), por su simpleza y el potencial paralelizable que tiene.
Es utilizado en la librería LAPACK como un algoritmo en dos pasos.

#### GOLUB - REINSCH

 - Se reduce la matriz A a una bidiagonal mediante una secuencia de transformaciones ortogonales de Householder.
 
<img src="Cap1.png" width="400">

 - Una vez bidiagonalizada la matriz A se hacen 0 los elementos que no están en la diagonal principal con un algoritmo que obtenga:
 
<img src="Cap2.png" width="250">

Donde $ Q_{S} $ y $ \Pi_{S} $ son matrices ortogonales.

 - La descomposición en valores singulares:
 
 <img src="Cap3.png" width="200">
 
 Donde:
 
 <img src="Cap4.png" width="200">
 
 
Para la segunda Fase:

 - Para obtener los valores singulares de la matriz bidiagonal $ B_{1} $, se procede iterativamente:

<img src="Cap5.png" width="150">

Se ogra mediante transformaciones hacia delante y hacia atrás iterativamente.

 - Se buscan transformaciones $U_{k}$ y $V_{k}$ ortogonales que reduzcan los valores sobre la diagonal de B.

Todas las iteraciones y actualizaciones del algoritmo pueden obtenerse mediante operaciones nivel 2 de la paquetería BLAS, con la inconveniencia de que computacionalmente la aproximación a la matriz por bloques es demasiado costoso.

Hasta este punto y considerando el algoritmo anterior la parte paralelizable será la de bidiagonalización y diagonalización de una matriz.

### Bidiagonalización en GPU:

PAra este problema en particular CUBLAS tiene funciones análogas a la paquetería estándar BLAS para realizar operaciones matriz - vector y matriz - matriz con un alto rendimiento computacional, preferentemente con dimensiones que sean múltiplos de 32 dados los arreglos de memoria.

Una gran ventaja de CUBLAS será que las operaciones necesarioas para las funciones de la paquetería son realizadas de forma local en el GPU, por lo cual no existe una transferencia de datos del CPU al GPU y no hay costo computacional en esto.

### Diagonalización en GPU 

En este algortimo si existe una transferencia de datos de los elementos diagonal y superdiagonal de la matriz B, mediante rotaciones de Givens secuencialmente se obtiene la diagonalizaciòn, pues solamente se requiere acceso a los elementos diagonales y superdiagonales de la matriz, por lo que el costo de transferencia es bajo, por lo que un enfoque mediante threads mejora el rendimiento en matrices de tamaño mayor.

Para la paralelización el algoritmo divide cada fila de la matriz en bloques, donde cada thread operara cada uno de los elementos iterativamente. Esto es facilmente implementable en CUDA dado que cada thread actúa independientemente y la información necesaria para cada bloque es almacenada en memoria compartida.

### Resultados:

El algoritmo se evalúa mediante optimización de CPU en MATLAB y un procesador Intel MLK y dos tarjetas de video NVIDIA: NVIDIA GeForce 280 GTX y NVIDIA Tesla S1070.

Se generaron 10 matrices aleatorias con números de precisión simple para cada entrada, el algoritmo de descomposición en valores singulares fue ejecutado 10 veces por matriz y se promedió sobre las matrices para evitar malas muestras.

Los resultados de las implementaciones son:

<img src="Cap6.png" width="350">

con los tiempos de ejecución:

<img src="Cap7.png" width="450">

<img src="Cap8.png" width="450">

### Conclusiones:

La implementación de los algoritmos de diagonalización y bidiagonalización de una matriz fueron implementados enteramente un una arquitectura CUDA utilizando a libreria CUBLAS para optimizar el rendimiento del equipo de cómputo.

Con esta implementación  se logró el procesamiento de matrices que un entorno tradicional de CPU no podría operar dadas las limitaciones de memoria con as que cuenta.
El error que se registró está en la vecindad del 0.001%, lo cual es aceptable.

Se ha demostrado con este ejercicio que el uso de paradigmas en paralelo para el cómputo obtiene beneficios a diferentes dimensiones dependiendo del problema o algoritmo a implementar, cuando se cuenta con un algoritmo estable numéricamente con posibilidades de paralelización altas se logran mejoras en tiempo de cómputo visibles con dimensionalidad relativamente baja, en este caso en el que contamos con dos procesos iterativos sucesivos que son paralelizables no es necesario contar con matrices de alta dimensionalidad para ver mejoras en el rendimiento, además de abrir la posibilidad a problemas de un tamaño mayor que de otra forma no podrían ser abordables en un entorno de cómputo tradicional.